In [2]:
import os
# import lxml
from lxml import etree
import pandas as pd
import numpy as np

In [3]:
import sys

In [22]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [4]:
import os

import gensim
import pymorphy2
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn import preprocessing

In [5]:
from sklearn.tree import DecisionTreeClassifier

In [6]:
from sklearn.preprocessing import Normalizer

In [27]:
cl = ['заметка','интервью','статья']

In [28]:
f = open('out.csv', 'r')

file_hash = {}
for line in f:
    attrs = line.split(';')
    path = attrs[0]
    class_name = attrs[8]
    if ' ' in class_name:
        class_name = class_name.split()[0]
    if '|' in class_name:
        class_name = class_name.split('|')[0]
    filename = os.path.basename(path)
    if class_name in cl:
        file_hash[filename] = class_name


In [29]:
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    return '{}_{}'.format(
        p.normal_form,
        p.tag.POS
    )

In [30]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            if filename in file_hash:
                class_text = file_hash[filename]
                tree = etree.parse(open_name)
                yield tree, class_text

In [31]:
sent = []
for tree, class_text in read_txt('/Users/Valeriya/Desktop/CoursePaper/corpus'):
    all_p = tree.xpath('//p')
    joined = '\n'.join([p.text for p in all_p if p.text is not None])
    sent.append((joined, class_text))

In [32]:
len(sent)

2676

In [33]:
only_texts = [text[0] for text in sent]
classes = [text[1] for text in sent]

In [14]:
m = 'ruwikiruscorpora_0_300_20.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

In [34]:
texts_norm = []
for text in sent:
    text_vectors = []
    words = text[0].split()
    for word in words:
        norm_word = normalize_and_add_pos(word)
        if norm_word in model:
            text_vectors.append(model[norm_word])
    mean_vector = sum(text_vectors) / len(text_vectors)
    texts_norm.append(mean_vector)

In [35]:
data = pd.DataFrame(texts_norm)
data['class'] = classes

In [36]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:300], data['class'], test_size=0.3)

# 1. Random Forest

In [37]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

    заметка       0.72      0.90      0.80       498
   интервью       0.28      0.15      0.19       107
     статья       0.61      0.36      0.45       198

avg / total       0.63      0.67      0.63       803



# 2. SVC

In [38]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print(classification_report(y_test, y_pred_svc))

             precision    recall  f1-score   support

    заметка       0.62      1.00      0.77       498
   интервью       0.00      0.00      0.00       107
     статья       0.00      0.00      0.00       198

avg / total       0.38      0.62      0.47       803



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 3. Logistic Regression

In [39]:
regr = LogisticRegression()
regr.fit(X_train, y_train)
y_pred_regr = regr.predict(X_test)
print(classification_report(y_test, y_pred_regr))

             precision    recall  f1-score   support

    заметка       0.63      0.98      0.77       498
   интервью       0.00      0.00      0.00       107
     статья       0.44      0.06      0.11       198

avg / total       0.50      0.63      0.50       803



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 4. OneVSRest

In [40]:
y_pred_rest = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).predict(X_test)
print(classification_report(y_test, y_pred_rest))

             precision    recall  f1-score   support

    заметка       0.67      0.94      0.78       498
   интервью       0.58      0.07      0.12       107
     статья       0.52      0.24      0.33       198

avg / total       0.62      0.65      0.58       803



# 5. SGD

In [42]:
clf_grad = linear_model.SGDClassifier()
clf_grad.fit(X_train, y_train)
y_pred_grad = clf_grad.predict(X_test)
print(classification_report(y_test, y_pred_grad))

             precision    recall  f1-score   support

    заметка       0.65      0.97      0.78       498
   интервью       0.40      0.02      0.04       107
     статья       0.54      0.15      0.23       198

avg / total       0.59      0.64      0.55       803



# 6. Bayes

In [43]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_norm = min_max_scaler.fit_transform(X_train)
X_test_norm = min_max_scaler.fit_transform(X_test)

In [44]:
clf_bayes = MultinomialNB()
clf_bayes.fit(X_train_norm, y_train)
y_pred_bayes = clf_bayes.predict(X_test_norm)
print(classification_report(y_test, y_pred_bayes))

             precision    recall  f1-score   support

    заметка       0.65      0.95      0.77       498
   интервью       0.40      0.02      0.04       107
     статья       0.41      0.15      0.22       198

avg / total       0.56      0.63      0.54       803



# 7. DecisionTree

In [45]:
clf_tree = DecisionTreeClassifier(random_state=0)
clf_tree.fit(X_train, y_train)
y_pred_tree = clf_tree.predict(X_test)
print(classification_report(y_test, y_pred_tree))

             precision    recall  f1-score   support

    заметка       0.75      0.73      0.74       498
   интервью       0.23      0.25      0.24       107
     статья       0.43      0.43      0.43       198

avg / total       0.60      0.60      0.60       803

